# Inference using the same validation dataset

## load validation dataset

## NPE Inference

## NPE-F Inference

## NLE Inference

## Analytical Inference

In [ ]:
import pymc3 as pm

# Define the model
with pm.Model() as model:
    # Define prior for 4 parameters
    param1 = pm.Uniform('param1', lower=a1, upper=b1) # Define appropriate bounds a1, b1
    param2 = pm.Uniform('param2', lower=a2, upper=b2) # Define appropriate bounds a2, b2
    param3 = pm.Uniform('param3', lower=a3, upper=b3) # Define appropriate bounds a3, b3
    param4 = pm.Uniform('param4', lower=a4, upper=b4) # Define appropriate bounds a4, b4

    # Define likelihood
    p = your_function(x, param1, param2, param3, param4) # Define your_function to take x and 4 parameters
    likelihood = pm.Bernoulli('obs', p=p, observed=y)

    # Run MCMC
    trace = pm.sample(5000)

# Analyze the results
pm.traceplot(trace)

In [ ]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

def model(x, y):
    # Priors for the 4 parameters
    param1 = pyro.sample("param1", dist.Uniform(a1, b1))
    param2 = pyro.sample("param2", dist.Uniform(a2, b2))
    param3 = pyro.sample("param3", dist.Uniform(a3, b3))
    param4 = pyro.sample("param4", dist.Uniform(a4, b4))

    # Define the likelihood function
    p = your_function(x, param1, param2, param3, param4) # Define your_function
    with pyro.plate("data", len(x)):
        pyro.sample("obs", dist.Bernoulli(p), obs=y)

# Convert x and y to PyTorch tensors
x_tensor = torch.tensor(x, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)

# Setup MCMC with NUTS kernel
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=5000, warmup_steps=1000)
mcmc.run(x_tensor, y_tensor)

# Get posterior samples
posterior_samples = mcmc.get_samples()
